# Агрегирование и группировка

In [1]:
import numpy as np
import pandas as pd

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
planets.head()

method  number  orbital_period   mass  distance  year
0  Radial Velocity       1         269.300   7.10     77.40  2006
1  Radial Velocity       1         874.774   2.21     56.95  2008
2  Radial Velocity       1         763.000   2.60     19.84  2011
3  Radial Velocity       1         326.030  19.40    110.62  2007
4  Radial Velocity       1         516.220  10.50    119.47  2009

In [4]:
rng = np.random.RandomState(1)
ser = pd.Series(rng.rand(5))
ser

0    0.417022
1    0.720324
2    0.000114
3    0.302333
4    0.146756
dtype: float64

In [5]:
ser.sum()

1.58654933641103

In [6]:
ser.mean()

0.317309867282206

In [7]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

A         B
0  0.092339  0.419195
1  0.186260  0.685220
2  0.345561  0.204452
3  0.396767  0.878117
4  0.538817  0.027388

In [8]:
df.mean()

A    0.311949
B    0.442874
dtype: float64

In [9]:
df.mean(axis='columns')

0    0.255767
1    0.435740
2    0.275006
3    0.637442
4    0.283102
dtype: float64

In [10]:
planets.dropna().describe()

number  orbital_period        mass    distance         year
count  498.00000      498.000000  498.000000  498.000000   498.000000
mean     1.73494      835.778671    2.509320   52.068213  2007.377510
std      1.17572     1469.128259    3.636274   46.596041     4.167284
min      1.00000        1.328300    0.003600    1.350000  1989.000000
25%      1.00000       38.272250    0.212500   24.497500  2005.000000
50%      1.00000      357.000000    1.245000   39.940000  2009.000000
75%      2.00000      999.600000    2.867500   59.332500  2011.000000
max      6.00000    17337.500000   25.000000  354.000000  2014.000000

Встроенные функции агрегирования

| Aggregation              | Description                     |
|--------------------------|---------------------------------|
| ``count()``              | Total number of items           |
| ``first()``, ``last()``  | First and last item             |
| ``mean()``, ``median()`` | Mean and median                 |
| ``min()``, ``max()``     | Minimum and maximum             |
| ``std()``, ``var()``     | Standard deviation and variance |
| ``mad()``                | Mean absolute deviation         |
| ``prod()``               | Product of all items            |
| ``sum()``                | Sum of all items                |

Применимы и к ``DataFrame`` и к ``Series``.

## GroupBy: Разбиение, применение, комбинация (split, apply, combine)

![](split-apply-combine.png)

In [11]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

key  data
0   A     0
1   B     1
2   C     2
3   A     3
4   B     4
5   C     5

In [15]:
df.groupby('key').max()

data
key      
A       3
B       4
C       5

In [16]:
df1 = pd.DataFrame({'key1': ['A', 'B', 'C', 'A', 'B', 'C'],'key2': ['A1', 'B1', 'C1', 'A1', 'B1', 'C1'],
                    'key3': ['A3', 'B3', 'C3', 'A3', 'B3', 'C3'],
                   'data': range(6),'data2': range(1,7)}, columns=['key1', 'key2', 'key3', 'data','data2'])

df1.groupby(['key1','key2']).sum()

data  data2
key1 key2             
A    A1       3      5
B    B1       5      7
C    C1       7      9

In [17]:
df1.groupby('key1').aggregate({'data': 'sum', 'data2': 'max'})

data  data2
key1             
A        3      4
B        5      5
C        7      6

Объект ``DataFrameGroupBy`` проводит вычисления "лениво":

In [18]:
df.groupby('key').sum()

data
key      
A       3
B       5
C       7

In [19]:
planets.groupby('method')

Этот объект можно рассматривать как словарь с ключем по колонкам

In [20]:
planets.groupby('method')['orbital_period']

In [21]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

#### Итерирование по группам

In [23]:
for (method, group) in planets.groupby('method'):
    print(f"{method:30} shape={group.shape}")

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


In [24]:
planets.groupby('method')['year'].describe()

count         mean       std     min      25%  \
method                                                                         
Astrometry                       2.0  2011.500000  2.121320  2010.0  2010.75   
Eclipse Timing Variations        9.0  2010.000000  1.414214  2008.0  2009.00   
Imaging                         38.0  2009.131579  2.781901  2004.0  2008.00   
Microlensing                    23.0  2009.782609  2.859697  2004.0  2008.00   
Orbital Brightness Modulation    3.0  2011.666667  1.154701  2011.0  2011.00   
Pulsar Timing                    5.0  1998.400000  8.384510  1992.0  1992.00   
Pulsation Timing Variations      1.0  2007.000000       NaN  2007.0  2007.00   
Radial Velocity                553.0  2007.518987  4.249052  1989.0  2005.00   
Transit                        397.0  2011.236776  2.077867  2002.0  2010.00   
Transit Timing Variations        4.0  2012.500000  1.290994  2011.0  2011.75   

                                  50%      75%     max  
method                                                  
Astrometry                     2011.5  2012.25  2013.0  
Eclipse Timing Variations      2010.0  2011.00  2012.0  
Imaging                        2009.0  2011.00  2013.0  
Microlensing                   2010.0  2012.00  2013.0  
Orbital Brightness Modulation  2011.0  2012.00  2013.0  
Pulsar Timing                  1994.0  2003.00  2011.0  
Pulsation Timing Variations    2007.0  2007.00  2007.0  
Radial Velocity                2009.0  2011.00  2014.0  
Transit                        2012.0  2013.00  2014.0  
Transit Timing Variations      2012.5  2013.25  2014.0

### Методы aggregate, filter, transform, apply

In [25]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

In [27]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [28]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

data1  data2
key              
A        0      5
B        1      7
C        2      9

In [29]:
def filter_func(x):
    return x['data2'].std() > 4

display(df, df.groupby('key').std(), df.groupby('key').filter(filter_func))

key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

In [30]:
df.groupby('key').transform(lambda x: x - x.mean())

data1  data2
0   -1.5    1.0
1   -1.5   -3.5
2   -1.5   -3.0
3    1.5   -1.0
4    1.5    3.5
5    1.5    3.0

In [31]:
def norm_by_data2(x):
    # x - это DataFrame сгруппированных значений!
    x['data1'] /= x['data2'].sum()
    return x

display(df, df.groupby('key').apply(norm_by_data2))

key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

key     data1  data2
0   A  0.000000      5
1   B  0.142857      0
2   C  0.166667      3
3   A  0.375000      3
4   B  0.571429      7
5   C  0.416667      9

### Определение ключа для разбиения

In [33]:
L = [0, 1, 0, 1, 2, 0]
display(df, df.groupby(L).sum())

key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

data1  data2
0      7     17
1      4      3
2      4      7

In [34]:
display(df, df.groupby(df['key']).sum())

key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

data1  data2
key              
A        3      8
B        5      7
C        7     12

In [36]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display(df2, df2.groupby(mapping).sum())

data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9

data1  data2
consonant     12     19
vowel          3      8

#### В разбиении можно использовать любую функцию Python

In [37]:
display(df2, df2.groupby(str.lower).mean())

data1  data2
key              
A        0      5
B        1      0
C        2      3
A        3      3
B        4      7
C        5      9

data1  data2
a    1.5    4.0
b    2.5    3.5
c    3.5    6.0

In [38]:
df2.groupby([str.lower, mapping]).mean()

data1  data2
a vowel        1.5    4.0
b consonant    2.5    3.5
c consonant    3.5    6.0

### Пример группировки

In [39]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade                         1980s  1990s  2000s  2010s
method                                                   
Astrometry                       0.0    0.0    0.0    2.0
Eclipse Timing Variations        0.0    0.0    5.0   10.0
Imaging                          0.0    0.0   29.0   21.0
Microlensing                     0.0    0.0   12.0   15.0
Orbital Brightness Modulation    0.0    0.0    0.0    5.0
Pulsar Timing                    0.0    9.0    1.0    1.0
Pulsation Timing Variations      0.0    0.0    1.0    0.0
Radial Velocity                  1.0   52.0  475.0  424.0
Transit                          0.0    0.0   64.0  712.0
Transit Timing Variations        0.0    0.0    0.0    9.0